# Monte Carlo Method
## Single Parameter

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

def compute_err(U, U_exact):
    n = len(U)
    err_inf = 0
    err_sq = 0
    for i in range(0,n):
        for j in range(0,n):
            err_sq += np.absolute(U_exact[i][j] - U[i][j])**2
            if np.absolute(U_exact[i][j] - U[i][j]) > err_inf:
                err_inf = np.absolute(U_exact[i][j] - U[i][j])
        
    err_sq = (err_sq * h**2)**0.5
    return err_inf, err_sq

def mcg_pre(T, F, tol):
    n = len(T)
    U = np.zeros([n,n])
    
    start_time = time.time()
    norm = np.sqrt(np.trace(np.matmul(T.transpose(),T)))
    alpha = -np.sqrt(n)/norm
    Y = np.linalg.inv((alpha*T - np.eye(n)))
    T_hat = np.eye(n) + 2*Y
    F_hat = 2*alpha*(np.matmul(np.matmul(Y,-F),Y))
    R = -F_hat + U - np.matmul(np.matmul(T_hat,U),T_hat)
    Q = np.matmul(np.matmul(T_hat.transpose(),R),T_hat.transpose()) - R
    Z = Q    

    num_iters = 0
#     if (np.any(np.absolute(Z) > tol).any()) == True:
    while ((np.absolute(R) > tol).any() == True):
        gamma = np.trace(np.matmul(R.transpose(),R))/np.trace(np.matmul(Z.transpose(),Z))
        eta_0 = np.trace(np.matmul(R.transpose(),R))
        U = U + gamma*Z
        R = -F_hat + U - np.matmul(np.matmul(T_hat,U),T_hat)
        Q = np.matmul(np.matmul(T_hat.transpose(),R),T_hat.transpose()) - R
        eta = np.trace(np.matmul(R.transpose(),R))/eta_0
        Z = Q + eta*Z
        num_iters += 1
    total_time = time.time() - start_time
    return U, total_time, num_iters

In [30]:
import numpy as np
from scipy.sparse import diags
import random
import time

def monte_carlo(r1, r2, M):
    
    random.seed(0)

    #Define mean, variance and solutions as zero matrices
    mean_sol = np.zeros([n,n])
    var_sol = np.zeros([n,n])
    
    
    sols = np.zeros([M,n,n])
        
    for i in range(0, M): 
        eps = np.random.uniform(r1, r2) #Generate random eps in range(r1,r2)
        T = np.multiply((-eps)/(h**2), diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())
        F = (2*(np.pi**2)*eps*np.sin(np.pi*X)*np.sin(np.pi*Y))+(34*(np.pi**2)*(eps**2)*np.sin(3*np.pi*X)*np.sin(5*np.pi*Y))
        U = mcg_pre(T,F,1e-4)[0]
        sols[i] = U
        mean_sol += U
        
    mean_sol = mean_sol/M #Divide by number of samples to get the mean solution
    
    return mean_sol
#     plot_sol(X,Y,mean_sol)
    
#     #Calculate variance
#     for i in range(0, M):
#         var_sol += ((sols[i] - mean_sol)**2)
#         var_sol = var_sol/(M-1)
    








In [32]:
import numpy as np
from scipy.sparse import diags
import sys

#Define parameters
n = 500
h = 1/(n+1) #step size

#Define x and y as arrays between 0 and 1 with n evenly spaced points (internal nodes)
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Create internal mesh (excludes boundaries)
X, Y = np.meshgrid(x, y, indexing='ij')

#Define tridiagonal matrix T
diagonals = [[-2],[1],[1]]

#Compute exact solution for comparison
U_exact = np.sin(np.pi*X) * np.sin(np.pi*Y) + 2*np.sin(3*np.pi*X)*np.sin(5*np.pi*Y)

# U = monte_carlo(1,3,40)

# err_inf, err_sq = compute_err(U, U_exact)
# print('Time taken:', total_time)
# print('Error inf:', err_inf, '\nError sq:' ,err_sq)

# U = monte_carlo(1,3,160)

# err_inf, err_sq = compute_err(U, U_exact)
# print('Time taken:', total_time)
# print('Error inf:', err_inf, '\nError sq:' ,err_sq)

U = monte_carlo(1,3,640)

err_inf, err_sq = compute_err(U, U_exact)
print('Time taken:', total_time)
print('Error inf:', err_inf, '\nError sq:' ,err_sq)

# U = monte_carlo(1,3,2560)

# err_inf, err_sq = compute_err(U, U_exact)
# print('Time taken:', total_time)
# print('Error inf:', err_inf, '\nError sq:' ,err_sq)



KeyboardInterrupt: 

### For SSH

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import diags
import random
import time
import sys

def compute_err(U, U_exact):
    n = len(U)
    err_inf = 0
    err_sq = 0
    for i in range(0,n):
        for j in range(0,n):
            err_sq += np.absolute(U_exact[i][j] - U[i][j])**2
            if np.absolute(U_exact[i][j] - U[i][j]) > err_inf:
                err_inf = np.absolute(U_exact[i][j] - U[i][j])
        
    err_sq = (err_sq * h**2)**0.5
    return err_inf, err_sq

def mcg_pre(T, F, tol):
    n = len(T)
    U = np.zeros([n,n])
    
    start_time = time.time()
    norm = np.sqrt(np.trace(np.matmul(T.transpose(),T)))
    alpha = -np.sqrt(n)/norm
    Y = np.linalg.inv((alpha*T - np.eye(n)))
    T_hat = np.eye(n) + 2*Y
    F_hat = 2*alpha*(np.matmul(np.matmul(Y,-F),Y))
    R = -F_hat + U - np.matmul(np.matmul(T_hat,U),T_hat)
    Q = np.matmul(np.matmul(T_hat.transpose(),R),T_hat.transpose()) - R
    Z = Q    

    num_iters = 0
#     if (np.any(np.absolute(Z) > tol).any()) == True:
    while ((np.absolute(R) > tol).any() == True):
        gamma = np.trace(np.matmul(R.transpose(),R))/np.trace(np.matmul(Z.transpose(),Z))
        eta_0 = np.trace(np.matmul(R.transpose(),R))
        U = U + gamma*Z
        R = -F_hat + U - np.matmul(np.matmul(T_hat,U),T_hat)
        Q = np.matmul(np.matmul(T_hat.transpose(),R),T_hat.transpose()) - R
        eta = np.trace(np.matmul(R.transpose(),R))/eta_0
        Z = Q + eta*Z
        num_iters += 1
    total_time = time.time() - start_time
    return U, total_time, num_iters

def monte_carlo(r1, r2, M, tol):
    
    random.seed(0)
    
    #Define mean as zero matrix
    mean_sol = np.zeros([n,n])
        
    start_time = time.time()
        
    for i in range(0, M): 
        eps = random.uniform(r1, r2) #Generate random eps in range(r1,r2)
        T = np.multiply((-eps)/(h**2), diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())
        F = (2*(np.pi**2)*eps*np.sin(np.pi*X)*np.sin(np.pi*Y))+(34*(np.pi**2)*(eps**2)*np.sin(3*np.pi*X)*np.sin(5*np.pi*Y))
        U = mcg_pre(T, F, tol=tol)[0]
        #sols[i] = U
        mean_sol += U
        
    mean_sol = mean_sol/M #Divide by number of samples to get the mean solution
    
    end_time = time.time()
    total_time = end_time - start_time
    
    return mean_sol, total_time

#Define parameters
n = int(sys.argv[1])
h = 1/(n+1) #step size

#Define x and y as arrays between 0 and 1 with n evenly spaced points (internal nodes)
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Create internal mesh (excludes boundaries)
X, Y = np.meshgrid(x, y, indexing='ij')

#Define tridiagonal matrix T
diagonals = [[-2],[1],[1]]

#Compute exact solution for comparison
U_exact = np.sin(np.pi*X) * np.sin(np.pi*Y) + 2*np.sin(3*np.pi*X)*np.sin(5*np.pi*Y)

M = int(sys.argv[2])
tol = float(sys.argv[3])

U, total_time = monte_carlo(1,3,M,tol)

err_inf, err_sq = compute_err(U, U_exact)
print('Time taken:', total_time)
print('Error inf:', err_inf, '\nError sq:' ,err_sq)

    






